<a href="https://colab.research.google.com/github/vifirsanova/synth-dataset/blob/main/ipa_annotation_stage1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Скрипт для попоплнения PHAT DATASET

PHAT: PHonological Annotation & Transcription

Наш репо на гитхабе (языковая модель, которую мы будем тестить): https://github.com/vifirsanova/phat-llm

Мобильное приложение, которое мы будем тестить: https://github.com/vifirsanova/TranscribePro

Базовая статья: TranscribePro: Enhancing linguistic analysis through mobile technology https://pureportal.spbu.ru/files/127361608/_021124.pdf

Куратор и автор проекта -- В.И. Фирсанова

ПО распространяется по лицензии MIT: все вопросы направляйте на vifirsanova@gmail.com

```
MIT License

Copyright (c) 2025 Victoria Firsanova

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

1. Берем текст из нашего бейзлайн-датасета

https://huggingface.co/datasets/missvector/multi-wiki-grammar

In [1]:
!pip install datasets -q # игнорим предупреждения Colab'а, это норма

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [2]:
from datasets import load_dataset

data = load_dataset('missvector/multi-wiki-grammar') # в датасете 400к строк, поэтому он будет грузиться достаточно долго, минуты 2

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train-00000-of-00011.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

train-00001-of-00011.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00002-of-00011.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

train-00003-of-00011.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

...

train-00007-of-00011.parquet:   0%|          | 0.00/154M [00:00<?, ?B/s]

train-00008-of-00011.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

train-00009-of-00011.parquet:   0%|          | 0.00/183M [00:00<?, ?B/s]

train-00010-of-00011.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/356534 [00:00<?, ? examples/s]

In [24]:
sample_text = data['train']['Text'][6][1515:3028] # в качестве образца беру 1 текст под индексом 5 (у вас будут свои индексы)
# обратите внимание, что я здесь беру отрывок в 1500 символов и режу его вручную так, чтобы получился цельный текст
sample_text

' Слоны — самые крупные наземные животные на Земле. Обитают они в Юго-Восточной Азии и Африке в тропических лесах и саваннах. Самыми крупными являются африканские саванные слоны, их масса может превышать 6 тонн.\nПолупальцеходящие животные. В центре стопы слона есть жировая подушка, которая каждый раз, когда слон опускает ногу, «расплющивается», увеличивая площадь опоры. На верхней челюсти у слонов есть бивни — видоизменённые резцы, растущие в течение всей жизни животного. Обычно слоны либо имеют два бивня, либо не имеют их вообще. С помощью бивней они обдирают кору с деревьев и рыхлят землю в поисках соли, а также наносят раны хищникам.\nЧтобы защититься от паразитов, слоны нередко обливаются жидкой грязью. Засохшая грязевая корочка служит хорошей защитой от насекомых и способствует заживлению ранок на коже.\nХарактерным органом слона является хобот, образованный из носа и верхней губы и заканчивающийся одним или двумя отростками. Расположенные на конце хобота ноздри служа

Важно! Выведите на экран ваши тексты заранее, занесите их в отдельный список для обработки, просмотрите их глазками

Если видите странные символы, мета-данные википедии, любые квадратные, фигурные скобки и т.д., обязательно проводим чистку с помощью регулярок

В конце текста будет список источников -- пускай будет, он внесет шум, но это не критично для решения нашей задачи

2. Делаем транскрипцию текста с помощью LLM

На текущий момент лучше использовать deepseek для разметки

In [29]:
!pip install huggingface_hub -q # коннетимся к хабу

from huggingface_hub import InferenceClient

TOKEN = '' # ваш токен
MODEL = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-32B' # не меняйте модель, пусть всю разметку делает именно эта версия

client = InferenceClient(MODEL, token=TOKEN) # инициализируем

# везде используем один системный промпт: можете потюнить для повышения точности, можете переключиться на русский
core_system = """generate IPA transcription for the provided text, return transcription only"""


# генеряем, оборачиваем в цикл for, чтобы не пробивать каждый текст вручную
# здесь я использую одноразовый запрос к дипсику
# заменяем sample_text на те данные, которые вы получили для разметки

# response_format не меняем, но проверьте, чтобы код выполнялся без ошибок
# если видите ошибки, меняем max_tokens, в частности, максимальная длина выдачи должна быть не меньше длины одного текста
# рекомендация: срезать текст из вики до ~1500 символов, установить max_tokens в 1500 токенов

# некоторые статьи очень длинные, они не поместятся в контекстное окно:
# подумайте, как поделить статьи на несколько отрывков, чтобы не терять дату (можно сделать это вручную по аналогии с примером про слонов)
# еще одна трудность: с HuggingFace API мы не можем генерять тексты очень долго, у нас временной лимит в 60 секунд
# обязательно сигнальте, если модель постоянно выдает ошибки, обсудим, как обрезать тексты и помещаться в лимиты контекстного окна и времени

# транскрипция будет генеряться около минуты
transcription = client.chat.completions.create(messages=[{"role": "system", "content": core_system},
                                                         {"role": "user", "content": sample_text},
                                                         ],
                                               response_format={"type": "json",
                                                                "value": {"properties": {"IPA": {"type": "string"},}}},
                                               stream=False,
                                               max_tokens=1500,
                                               temperature=0.7,
                                               top_p=0.1
                                               ).choices[0].get('message')['content']

In [30]:
transcription # качество транскрипции нужно будет отсмотреть, но мы потом поставим на это отдельную задачу

'{ "IPA": "ˈsloːnə səˈmaːjə kruːpʲɪjə nɐzʲɪmʲɪsʲkʲɪjə zhoʊˈvʲotnə sə zʲemʲlʲe. Obʲɪtajətʲɵnʲi jɪ vʲuˈɡo-vʲesʲtʲɪkʲɪjə ˈazʲjɪ i ˈafʲrɪkʲe v trəˈpʲɪtsʲɪkʲɪjə lʲesʲi i səˈvanəx. Səˈmaːjə kruːpʲɪjə jəsʲtʲɵ ˈafʲrɪkənsʲkʲi səˈvanəsʲkʲi ˈsloːnə, jɪr məˈsʲa məˈɡʲi prʲeˈvʲɪʂətʲɵ 6 tɔn. Polupalʲtsohodʲiˈskʲi zhoʊˈvʲotnə. V tsʲentʲrə stɔpə slənə jɛstʲɵ ˈʐyɪvəjə pəˈdʊʂkə, kətərəjə kəzʲdə rəz, kəɡda slən ˈopuskətʲɵ nɔɡu, ˈraspluːsʲkətsə, uˈklʲiːnʲiˈvəjə pʲləˈsʲɪtʲsʲi ˈopory. Na vʲerxəjə ˈtɕelʲjʊstʲi u slənəv jɛstʲɵ ˈbʲivnʲi — vʲidəuzmʲɛnʲɪvərə ˈrʲeːztsɨ, rəstjʊˈʃtɨ vʲe tʲolʲkə ˈzɨtʲɵ ˈzhɨvətʲɵ. Obyːdʲiˈnə slənə lʲiːbo jɪmətʲɵ dva bʲivnʲi, lʲiːbo ne jɪmətʲɵ jɪvə ˈɡʲenʲu. Sə poməskʲu bʲivnʲi slənə ˈobdʲirətʲɵ kɔrə s dʲerʲev, i rɨxʲlʲatʲɵ zʲemʲlu v pəˈsʲɪx kɔli, a təmʲe ˈnənəsʲtʲɵ rənɨ ˈkhiːsʲtʲi. Čtəbɨ zəshʲitʲɵ sə pərəzʲitəs, slənə nərʲeːdə ˈobliːvətʲɵ ˈzhidʲuˈjə ˈɡrədzʲu. Zasəjkʲuˈsʲkə ˈgrədzʲevəjə kɔrətʃkə słuːžitʲɵ xərəˈskəjə zəshʲitʲɵ ˈot nəsʲkəkʲi i səbəzʲdʲuˈetʲɵ zʲaʐivlʲenʲiye rənək ˈna kəzʲi

3. Упаковываем в датасет, чтобы не потерять данные потом и сразу отправляем на вашу ветку в https://github.com/vifirsanova/synth-dataset

In [32]:
import pandas as pd
import uuid
import numpy as np

df = pd.DataFrame({
    'ID': [uuid.uuid4()], # генеряем уникальный ID для каждого текста
    'text': [sample_text], # все тексты, которые вы обработали
    'IPA': [transcription], # транскрипция в порядке текстов
    'audio': [np.nan], # для каждого текста пока создаем плейсхолдер (нулевое значение NaN), генерять аудио будем отдельным скриптом, чтобы не потерять данные
    'pauses': [np.nan], # то же самое, проставляем везде NaN, для пауз будем использовать специальное ПО
})

In [33]:
df

,ID,text,IPA,audio,pauses
0,9ebaea2d-1347-4715-bf45-14aa87f1790e,Слоны — самые крупные наземные животные на Зе...,"{ ""IPA"": ""ˈsloːnə səˈmaːjə kruːpʲɪjə nɐzʲɪmʲɪs...",NaN,NaN


Сохраняем и кидаем на свою ветку

In [34]:
df.to_csv('wiki.csv')